In [1]:
# ON GOOGLE COLAB, THIS SECTION MUST BE INITIALIZED. IT TAKES ABOUT 2MINUTES.
# AFTER RUNNING THIS SECTION, RUNTIME(SESSION) MUST BE RESTARTED, IN ORDER TO
# USE labwons/ PACKAGE. SESSION RESTART HOTKEY IS CTRL + M.
import os
if any("COLAB" in e for e in os.environ):
    from google.colab import drive
    drive.mount('/content/drive')

    !git clone https://github.com/labwons/labwons.git
    %cd labwons
    !pip install -r requirements.txt -e .

Mounted at /content/drive
Cloning into 'labwons'...
remote: Enumerating objects: 20850, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 20850 (delta 129), reused 278 (delta 82), pack-reused 20499 (from 3)
Receiving objects: 100% (20850/20850), 584.22 MiB | 24.75 MiB/s, done.
Resolving deltas: 100% (12007/12007), done.
/content/labwons
Obtaining file:///content/labwons
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
JOBS   = [
    # "FETCH.DAILY-MARKET",
    "FETCH.SECTOR-COMPOSITION",
    # "FETCH.FINANCIAL-STATEMENT",
    # "FETCH.STOCK-DATA"
]

In [2]:
from labwons.util import DATETIME
import os

os.chdir(os.environ.get("GITHUB_WORKSPACE", os.getcwd()))
ACTION = os.environ.get("GITHUB_EVENT_NAME", "LOCALHOST").upper()
HOSTID = os.environ.get("USERDOMAIN", "COLAB") if ACTION == "LOCALHOST" else "GITHUB"

# GITHUB ACTION
if ACTION == "SCHEDULE":

    # SYSTEM EXITS WHEN THE LATEST TRADING DATE AND BUILD DATE IS DIFFERENT.
    # THIS CODE IS IMPLEMENTED IN ORDER TO AVOID RUNNING ON WEEKDAY OR HOLIDAY
    # OF THE MARKET.
    if DATETIME.TRADING != DATETIME.TODAY:
        raise SystemExit

    # IF THE TASK IS TRIGGERED BEFORE THE MARKET CLOSES, WHICH HARDLY HAPPENS,
    # SYSTEM PAUSES UNTIL THE MARKET CLOSES
    while (DATETIME.TIME().hour == 15) and (15 <= DATETIME.TIME().minute < 31):
        DATETIME.pause(30)

    if DATETIME.TIME().hour >= 20:
        JOBS = ["FETCH.ECOS", "FETCH.FRED", "FETCH.FINANCIAL-STATEMENT"]
    else:
        JOBS = ["FETCH.DAILY-MARKET", "FETCH.STOCKS"]
    JOBS += ["BUILD.MARKET-BASELINE", "BUILD.MACRO-BASELINE", "DEPLOY.ALL"]

# if ACTION == "LOCALHOST" and HOSTID == "HKEFICO":
#     JOBS = [job for job in JOBS if not job.startswith("FETCH")]

In [4]:
from labwons.path import ARCHIVE, GITHUB_REMOTE
from labwons.fetch import (
    MarketDaily, MarketSectors, FinancialStatement
)

ARCHIVE.create(DATETIME.TODAY)
if "FETCH.DAILY-MARKET" in JOBS:
    MarketDaily.fetch(ARCHIVE.MARKET_DAILY)

if "FETCH.SECTOR-COMPOSITION" in JOBS:
    MarketSectors.fetch(ARCHIVE.MARKET_SECTORS)

if "FETCH.FINANCIAL-STATEMENT" in JOBS:
    from pandas import read_parquet
    tickers = read_parquet(ARCHIVE.recentBaseline, engine='pyarrow').index
    financialStatement = FinancialStatement(*tickers)
    financialStatement.fetchOverview(ARCHIVE.MARKET_OVERVIEW)
    financialStatement.fetchAnnualStatement(ARCHIVE.STATEMENT_A)
    financialStatement.fetchQuarterStatement(ARCHIVE.STATEMENT_Q)

if "FETCH.STOCK-DATA" in JOBS:
    # TODO
    # 로컬(HKEFICO)에서 개발할 때 사용하는 Cache
    # 실제로는 즉시 Fetch -> Deploy 로 사용
    pass

if HOSTID == "COLAB" and "FETCH.SECTOR-COMPOSITION" in JOBS:
    import json
    with open(r"/content/drive/MyDrive/secrets.json") as secrets:
        os.environ.update(json.load(secrets))
    %cd labwons
    !git config --global user.name "$GITHUB_USER"
    !git config --global user.email "$GITHUB_EMAIL"
    !git remote set-url origin "https://$GITHUB_TOKEN@github.com/$GITHUB_REPO/$GITHUB_REPO.git"
    !git add {ARCHIVE.MARKET_SECTORS}
    !git commit -m "UPDATE FROM COLAB"
    !git push origin main

# TODO
# 일정 기간이 지난 날짜의 아카이브 폴더는 드라이브로 백업하는 코드 작성: Clean-UP

2025-08-26 09:07:49 [FETCH] RUN [FETCH SECTOR COMPOSITION]
2025-08-26 09:07:54 [FETCH] - 01 / 27 : WI100 에너지
2025-08-26 09:07:55 [FETCH] - 02 / 27 : WI110 화학
2025-08-26 09:07:57 [FETCH] - 03 / 27 : WI200 비철금속
2025-08-26 09:07:58 [FETCH] - 04 / 27 : WI210 철강
2025-08-26 09:08:00 [FETCH] - 05 / 27 : WI220 건설
2025-08-26 09:08:02 [FETCH] - 06 / 27 : WI230 기계
2025-08-26 09:08:04 [FETCH] - 07 / 27 : WI240 조선
2025-08-26 09:08:06 [FETCH] - 08 / 27 : WI250 상사,자본재
2025-08-26 09:08:07 [FETCH] - 09 / 27 : WI260 운송
2025-08-26 09:08:09 [FETCH] - 10 / 27 : WI300 자동차
2025-08-26 09:08:10 [FETCH] - 11 / 27 : WI310 화장품,의류
2025-08-26 09:08:12 [FETCH] - 12 / 27 : WI320 호텔,레저
2025-08-26 09:08:13 [FETCH] - 13 / 27 : WI330 미디어,교육
2025-08-26 09:08:15 [FETCH] - 14 / 27 : WI340 소매(유통)
2025-08-26 09:08:17 [FETCH] - 15 / 27 : WI400 필수소비재
2025-08-26 09:08:19 [FETCH] - 16 / 27 : WI410 건강관리
2025-08-26 09:08:22 [FETCH] - 17 / 27 : WI500 은행
2025-08-26 09:08:24 [FETCH] - 18 / 27 : WI510 증권
2025-08-26 09:08:26 [FETCH] - 1

In [ ]:
if HOSTID == "GITHUB":
    from labwons.util import Mail
    from labwons.logs import read_log

    report = Mail()
    report.content = content = read_log("fetch", "build")
    report.subject = f'[{"FAILED" if "FAILED" in content else "SUCCESS"}] BUILD LABWONS : {DATETIME.CLOCK().strftime("%Y/%m/%d %H:%M")}'
    report.send()
    # print(content) # TODO 만약 로그가 GITHUB 콘솔에 찍히면 삭제하세요.

'\n\n'